In [50]:
import pandas as pd
import numpy as np
import re
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn import svm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [23]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [51]:
## Store the usefull information from this huge dataset
dataset = {}
dataset["review_text"] = []
dataset['rating'] = []
count = 0
with open ('Books_5.json') as Train_json:
    for i in Train_json:
        count+=1
        if count % 1 == 0:
            item = json.loads(i)
            dataset["review_text"].append(item["reviewText"])
            dataset["rating"].append(item["overall"])

In [ ]:
print(1)

In [ ]:
## Convert Dataset to dataframe
dataset_df = pd.DataFrame(dataset)
print(len(dataset_df))
dataset_df.head()

In [4]:
dataset_df_5 = dataset_df[dataset_df["rating"]==5.0]
dataset_df_4 = dataset_df[dataset_df["rating"]==4.0]
dataset_df_3 = dataset_df[dataset_df["rating"]==3.0]
dataset_df_2 = dataset_df[dataset_df["rating"]==2.0]
dataset_df_1 = dataset_df[dataset_df["rating"]==1.0]

In [5]:
dataset_df_1_train = dataset_df_1.iloc[0:5000,]
dataset_df_1_test = dataset_df_1.iloc[5000:6000,]
dataset_df_2_train = dataset_df_2.iloc[0:5000,]
dataset_df_2_test = dataset_df_2.iloc[5000:6000,]
dataset_df_3_train = dataset_df_3.iloc[0:5000,]
dataset_df_3_test = dataset_df_3.iloc[5000:6000,]
dataset_df_4_train = dataset_df_4.iloc[0:5000,]
dataset_df_4_test = dataset_df_4.iloc[5000:6000,]
dataset_df_5_train = dataset_df_5.iloc[0:5000,]
dataset_df_5_test = dataset_df_5.iloc[5000:6000,]

In [6]:
frames_train = [dataset_df_5_train, dataset_df_4_train, dataset_df_3_train,dataset_df_2_train,dataset_df_1_train]
frames_test = [dataset_df_5_test, dataset_df_4_test, dataset_df_3_test,dataset_df_2_test,dataset_df_1_test]

dataset_df_train = pd.concat(frames_train)
dataset_df_test = pd.concat(frames_test)

In [7]:
## Clean the reviews
def data_cleaning (raw_review_str):
    ## remove potential HTML tag
    review_text = BeautifulSoup(raw_review_str).get_text()
    ## remove non-letter 
    review_letter = re.sub("[^a-zA-Z]"," ",review_text)
    ## convert to lower case and split the list
    review_lower_list = review_letter.lower().split()
    ## remove stop words and stemming
    stops_eng = set(stopwords.words("english"))
    stemmer = nltk.wordnet.WordNetLemmatizer()
    #review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list]
    review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list if w not in stops_eng]
    ## return 
    return " ".join(review_nostopwords_stemmed)

In [8]:
dataset_df_train["review_cleaned"] = dataset_df_train["review_text"].apply(data_cleaning)
dataset_df_test["review_cleaned"] = dataset_df_test["review_text"].apply(data_cleaning)

/Users/xinmu/anaconda/envs/py36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/xinmu/anaconda/envs/py36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


In [9]:
## Put all the review of train_set in a list to generate the features
clean_train_reviews = []
for i in dataset_df_train["review_cleaned"]:
    clean_train_reviews.append(i)

clean_test_reviews = []
for i in dataset_df_test["review_cleaned"]:
    clean_test_reviews.append(i) 

In [36]:
clean_total_reviews = clean_train_reviews + clean_test_reviews

In [37]:
tagged_data_total = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(clean_total_reviews)]

In [38]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

'''Note: dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM) and 
dm =0 means ‘distributed bag of words’ (PV-DBOW). 
Distributed Memory model preserves the word order in a document whereas 
Distributed Bag of words just uses the bag of words approach, 
which doesn’t preserve any word order.'''
  
model.build_vocab(tagged_data_total)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data_total,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [39]:
#model= Doc2Vec.load("d2v.model")
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('17372', 0.8125935196876526), ('1860', 0.8093466758728027), ('5537', 0.8002928495407104), ('3520', 0.7915164232254028), ('23834', 0.7913970947265625), ('551', 0.7830069065093994), ('23946', 0.7801980972290039), ('3559', 0.7773354053497314), ('3821', 0.7741789817810059), ('1304', 0.7734559774398804)]


In [44]:
# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['0'])
print(type(model.docvecs['0']))
print(model.docvecs['0'].shape)

[ -5.09856272  -7.57122469   0.33858123   6.5491128   -2.72524428
   6.56191778  -3.8429153   -4.76670647  -1.68930018   1.14285815
   3.92246819  -5.80684757  -0.44117892  -5.36924076  -6.65282536
  -0.06839881  -5.58715105  10.39065075  -0.3934164   -2.91169906]
<class 'numpy.ndarray'>
(20,)


In [45]:
clean_train_reviews_vec = []
for sen in clean_train_reviews:
    train_data = word_tokenize(sen)
    clean_train_reviews_vec.append(model.infer_vector(train_data))

In [47]:
print(len(clean_train_reviews_vec))
print(clean_train_reviews_vec[0])

25000
[ 0.02122341 -0.73725897  0.10229225  0.3133432  -0.04938582  0.08933731
 -0.35031724 -1.52035999 -0.90688658 -0.04850008 -0.3458958  -0.61721355
  0.05665984 -1.04439497 -0.25586617 -0.25951532 -0.10752334  1.39745378
 -0.66864818 -0.3141073 ]


In [48]:
clean_test_reviews_vec = []
for sen in clean_test_reviews:
    test_data = word_tokenize(sen)
    clean_test_reviews_vec.append(model.infer_vector(test_data))

In [ ]:
#